- [间隔度量](#间隔度量)
- [线性可分支持向量机——硬间隔最大化](#线性可分支持向量机——硬间隔最大化)
  - [对偶求解](#对偶求解)
  - [推导求解](#推导求解)
  - [支持向量](#支持向量)
- [线性支持向量机——软间隔最大](#线性支持向量机——软间隔最大)
  - [推导求解](#推导求解)
  - [软间隔支持向量](#软间隔支持向量)
- [非线性支持向量分类机与核函数](#非线性支持向量分类机与核函数)
- [序列最小最优化算法（SMO）](#序列最小最优化算法（SMO）)
  - [二次规划求解法](#二次规划求解法)
  - [变量选择方法](#变量选择方法)
- [算法实现](#算法实现)

支持向量机是二分类模型，把问题转换为凸二次规划（也可能不需要转换）并求最优（间隔最大）

直接可分——线性可分支持向量机——硬间隔最大

近似线性可分——线性支持向量机——软间隔最大

线性不可分——非线性支持向量机——核技巧+软间隔最大

核技巧是借助核函数，隐式的将输入从低维不可线性分隔的输入空间映射到高维可线性分隔的特征空间进行线性分隔。同时不用求高维空间的向量表示，核函数本身的映射结果就是高维空间内的向量积。

# 间隔度量

回顾感知机模型，线性可分就是存在超平面 $w \cdot x+b=0$。 能将正实例和负实例完全、正确地划分到超平面两侧，即对所有 $y=+1$ 的样本有 $w \cdot x+b > 0$，$y=-1$ 的样本有 $w \cdot x+b < 0$，即
$$
y(w \cdot x+b)>0
$$
支持向量机在此基础上要求间隔最大，而 $|w \cdot x+b|$ 在 $w,b$ 确认的情况下就能表示距离超平面的远近。所以用 $\hat{\gamma}_{i}=y_{i}\left(w \cdot x_{i}+b\right)$ 表示数据集中各点的**函数间隔**，整个样本集的函数间隔取其最小值
$$
\hat{\gamma}=\min _{i=1, \cdots, N} \hat{\gamma}_{i}
$$
函数间隔不稳定，$w,b$ 扩大二倍，它也跟着扩大了二倍。对法向量规范化，变为单位向量，得到**几何间隔**
$$
\gamma_{i}=y_{i}\left(\frac{w}{\|w\|} \cdot x_{i}+\frac{b}{\|w\|}\right)\\
\gamma=\min _{i=1, \cdots, N} \gamma_i
$$
可见两种间隔只差了规范化系数 $\frac{1}{\|w\|}$

# 线性可分支持向量机——硬间隔最大化

![](https://raw.githubusercontent.com/LibertyDream/diy_img_host/master/img/2019-11-27_svm_maximum_margin.png)

线性可分（“肉眼可分”），就求几何间隔最大，所以问题描述成
$$
\begin{array}{ll}{\max _{w, b}} & {\gamma} \\ {\text { s.t. }} & {y_{i}\left(\frac{w}{\|w\|} \cdot x_{i}+\frac{b}{\|w\|}\right) \geqslant \gamma, \quad i=1,2, \cdots, N}\end{array}
$$
考虑到两种间隔间的关系，改写成
$$
\begin{array}{ll}{\max _{w . b}} & {\frac{\hat{\gamma}}{\|w\|}} \\ {\text { s.t. }} & {y_{i}\left(w \cdot x_{i}+b\right) \geqslant \hat{\gamma}, \quad i=1,2, \cdots, N}\end{array}
$$
注意到 $\hat{\gamma}$ 对不等式没影响，需要的只是最小值，具体多小无所谓。所以取 $\hat{\gamma}=1$，此时变成了最大化 $\frac{1}{\|w\|}$，这和最小化 $\frac{1}{2}\|w\|^{2}$ 等价，于是得到了**线性可分支持向量机要解的最优化问题**
$$
\begin{array}{cl}{\min _{w, b}} & {\frac{1}{2}\|w\|^{2}} \\ {\text { s.t. }} & {y_{i}\left(w \cdot x_{i}+b\right)-1 \geqslant 0, \quad i=1,2, \cdots, N}\end{array}
$$

>凸优化指约束下的最优化问题
>$$
>\begin{array}{ll}{\min _{w}} & {f(w)} \\ {\text { s.t. }} & {g_{i}(w) \leqslant 0, \quad i=1,2, \cdots, k} \\ {} & {h_{i}(w)=0, \quad i=1,2, \cdots, l}\end{array}
>$$
>$f(w)$ 和 $g(w)$ 在是 $\mathbf{R}^{n}$ 上连续可导的凸函数，$h(w)$ 是 $\mathbf{R}^{n}$ 上的仿射函数。如果 $f(w)$ 是二次函数， $g(w)$ 为仿射函数，此时的凸优化叫**凸二次规划问题**
>
>凸函数：凸集上，割线在上方的函数。$f(\theta x+(1-\theta) y) \leqslant \theta f(x)+(1-\theta) f(y)$
>
>仿射函数：满足该式 $f(x)=a \cdot x+b, a \in \mathbf{R}^{n}, b \in \mathbf{R}, x \in \mathbf{R}^{n}$
>
>凸集：连接两点的线段上的点都在一个定义域内
>
>仿射集：连接两点直线上的点都在一个定义域内

## 对偶求解

带约束的凸优化问题的一般套路是使用拉格朗日乘数法转换成无约束最优化问题（就可以开心的求导了），引入拉格朗日乘子 $\alpha_i \geqslant 0$，给目标函数“加个 0”
$$
L(x, \alpha, \beta)=f(x)+\sum_{i=1}^{k} \alpha_{i} g_{i}(x)+\sum_{j=1}^{l} \beta_{j} h_{j}(x)
$$
具体到这里就是
$$
L(w, b, \alpha)=\frac{1}{2}\|w\|^{2}-\sum_{i=1}^{N} \alpha_{i} y_{i}\left(w \cdot x_{i}+b\right)+\sum_{i=1}^{N} \alpha_{i}
$$
如果求 $L$ 的最大值，只有满足原始约束 $L$ 才退化为 $f$ ，所以就有
$$
\min _{w,b} f(w,b)=\min _{w,b} \max _{\alpha, \beta: \alpha_{i} \geqslant 0} L(w,b, \alpha)
$$
这叫**原始问题**，他有个兄弟**对偶问题**  $\max _{\alpha, \beta: \alpha_{i} \geqslant 0}\min _{w,b}  L(w,b, \alpha)$，当二者都有最优值时，有
$$
d^{*}=\max _{\alpha, \beta: \alpha_{i} \geqslant 0} \min _{x} L(x, \alpha, \beta) \leqslant \min _{x} \max _{\alpha, \beta: \alpha_{i} \geqslant 0} L(x, \alpha, \beta)=p^{*}
$$
如果满足三个条件， $f,g$ 是凸函数，$h$ 为仿射函数，$g_{i}(x) \leqslant 0$ 严格可行，即 $\exists x，\forall g_{i}(x) < 0$，则 $p^{*}=d^{*}=L\left(x^{*}, \alpha^{*}, \beta^{*}\right)$

而在三个条件下，$x^{*}, \alpha^{*}, \beta^{*}$ 同时是两个问题的解的充要条件称为**KKT条件**：
$$
\begin{array}{rl}{\nabla_{x} L\left(x^{*}, \alpha^{*}, \beta^{*}\right)} & {=0} \\ {\alpha_{i}^{*} g_{i}\left(x^{*}\right)=0,} & {i=1,2, \cdots, k} \\ {g_{i}\left(x^{*}\right) \leqslant 0,} & {i=1,2, \cdots, k} \\ {\alpha_{i}^{*} \geqslant 0,} & {i=1,2, \cdots, k} \\ {h_{j}\left(x^{*}\right)=0} & {j=1,2, \cdots, l}\end{array}
$$
其中第二条又叫 KKT 的对偶互补条件，若知 $\alpha_{i}^{*}>0$，则 $g_{i}\left(x^{*}\right)=0$

支持向量机的最优化问题一般都会转为求解对偶问题的最优值，优点有二：

- 对偶问题往往更简单
- 自然引入核函数，推广到非线性分类

## 推导求解

$$
L(w, b, \alpha)=\frac{1}{2}\|w\|^{2}-\sum_{i=1}^{N} \alpha_{i} y_{i}\left(w \cdot x_{i}+b\right)+\sum_{i=1}^{N} \alpha_{i}
$$

- 求 $\min _{w,b} L(w,b, \alpha)$

$$
\begin{aligned} \nabla_{w} L(w, b, \alpha) &=w-\sum_{i=1}^{N} \alpha_{i} y_{i} x_{i}=0 \\ \nabla_{b} L(w, b, \alpha) &=-\sum_{i=1}^{N} \alpha_{i} y_{i}=0 \end{aligned}\\
w=\sum_{i=1}^{N} \alpha_{i} y_{i} x_{i}\\
\sum_{i=1}^{N} \alpha_{i} y_{i}=0
$$

带入 $L$
$$
\begin{aligned} L(w, b, \alpha) &=\frac{1}{2} \sum_{i=1}^{N} \sum_{j=1}^{N} \alpha_{i} \alpha_{j} y_{i} y_{j}\left(x_{i} \cdot x_{j}\right)-\sum_{i=1}^{N} \alpha_{i} y_{i}\left(\left(\sum_{j=1}^{N} \alpha_{j} y_{j} x_{j}\right) \cdot x_{i}+b\right)+\sum_{i=1}^{N} \alpha_{i} \\ &=-\frac{1}{2} \sum_{i=1}^{N} \sum_{j=1}^{N} \alpha_{i} \alpha_{j} y_{i} y_{j}\left(x_{i} \cdot x_{j}\right)+\sum_{i=1}^{N} \alpha_{i}
\end{aligned}\\
\min _{w, b} L(w, b, \alpha)=-\frac{1}{2} \sum_{i=1}^{N} \sum_{j=1}^{N} \alpha_{i} \alpha_{j} y_{i} y_{j}\left(x_{i} \cdot x_{j}\right)+\sum_{i=1}^{N} \alpha_{i}
$$

- 对 $\alpha$ 求极大

取个负号变成求极小，成为对偶最优化问题
$$
\begin{array}{ll}{\min _{\alpha}} & {\frac{1}{2} \sum_{i=1}^{N} \sum_{j=1}^{N} \alpha_{i} \alpha_{j} y_{i} y_{j}\left(x_{i} \cdot x_{j}\right)-\sum_{i=1}^{N} \alpha_{i}} \\ {\text { s.t. }} & {\sum_{i=1}^{N} \alpha_{i} y_{i}=0} \\ {} & {\alpha_{i} \geqslant 0, \quad i=1,2, \cdots, N}\end{array}
$$
得最优解$\alpha^{*}=\left(\alpha_{1}^{*}, \alpha_{2}^{*}, \cdots, \alpha_{N}^{*}\right)^{\mathrm{T}}$

原始问题满足三个条件，故两问题存在相同最优解，。可验证 KKT 条件成立
$$
\begin{aligned} &\nabla_{w} L\left(w^{*}, b^{*}, \alpha^{*}\right) =w^{*}-\sum_{i=1}^{N} \alpha_{i}^{*} y_{i} x_{i}=0 \\ 
&\nabla_{b} L\left(w^{*}, b^{*}, \alpha^{*}\right) =-\sum_{i=1}^{N} \alpha_{i}^{*} y_{i}=0 \\ 
&\alpha_{i}^{*}\left(y_{i}\left(w^{*} \cdot x_{i}+b^{*}\right)-1\right)=0,  i=1,2, \cdots, N \\
&y_{i}\left(w^{*} \cdot x_{i}+b^{*}\right)-1 \geqslant 0, i=1,2, \cdots, N \\ & \alpha_{i}^{*} \geqslant 0, \quad i=1,2, \cdots, N \end{aligned}
$$
得 $w^{*}=\sum_{i} \alpha_{i}^{*} y_{i} x_{i}$

如果 $\alpha^*=0$，则 $w^*=  0$，而对于线性可分数据集，因为正负类点都存在 $(w,b)=(0,b)$ 不会是可行解，最优解$(w^*,b^*)$ 必满足$w^* \neq 0$，所以至少有一个 $\alpha_{j}^{*} > 0$，有 $y_{j}\left(w^{*} \cdot x_{j}+b^{*}\right)-1=0$，而$y_{j}^{2}=1$
$$
b^{*}=y_{j}-\sum_{i=1}^{N} \alpha_{i}^{*} y_{i}\left(x_{i} \cdot x_{j}\right)
$$

- 结果

分离超平面
$$
w^{*} \cdot x+b^{*}=\sum_{i=1}^{N} \alpha_{i}^{*} y_{i}\left(x \cdot x_{i}\right)+b^{*}=0
$$
分类决策函数
$$
f(x)=\operatorname{sign}\left(w^{*} \cdot x+b^{*}\right)=\operatorname{sign}\left(\sum_{i=1}^{N} \alpha_{i}^{*} y_{i}\left(x \cdot x_{i}\right)+b^{*}\right)
$$

## 支持向量

$a_i^*>0$ 的样本点就是支持向量，因为 $\alpha_{i}^{*}\left(y_{i}\left(w^{*} \cdot x_{i}+b^{*}\right)-1\right)=0$，所以$w^{*} \cdot x_{i}+b^{*}=\pm 1$，即支持向量在间隔边界上

# 线性支持向量机——软间隔最大

“肉眼可分”的分布形式总是稀少的，多数情况下都会有一些样本跑到边界里面

![](https://raw.githubusercontent.com/LibertyDream/diy_img_host/master/img/2019-11-28_svm_soft_margin.png)

所以加入松弛变量，保证函数间隔加上松弛变量后距离大于等于1
$$
\begin{array}{ll}{\min _{w, b, \xi}} & {\frac{1}{2}\|w\|^{2}+C \sum_{i=1}^{N} \xi_{i}} \\ {\text { s.t. }} & {y_{i}\left(w \cdot x_{i}+b\right) \geqslant 1-\xi_{i}, \quad i=1,2, \cdots, N} \\ {} & {\xi_{i} \geqslant 0, \quad i=1,2, \cdots, N}\end{array}
$$
$C > 0$是惩罚参数，目标函数前面是保证间隔尽量大，后面是分类错的尽量少，$C$ 是调和二者关系的

## 推导求解

- 极小

$$
L(w, b, \xi, \alpha, \mu) \equiv \frac{1}{2}\|w\|^{2}+C \sum_{i=1}^{N} \xi_{i}-\sum_{i=1}^{N} \alpha_{i}\left(y_{i}\left(w \cdot x_{i}+b\right)-1+\xi_{i}\right)-\sum_{i=1}^{N} \mu_{i} \xi_{i}\\
\alpha_i \geqslant 0,\mu_i \geqslant 0
$$

$$
\begin{array}{c}{\nabla_{w} L(w, b, \xi, \alpha, \mu)=w-\sum_{i=1}^{N} \alpha_{i} y_{i} x_{i}=0} \\ {\nabla_{b} L(w, b, \xi, \alpha, \mu)=-\sum_{i=1}^{N} \alpha_{i} y_{i}=0} \\ {\nabla_{\xi_{i}} L(w, b, \xi, \alpha, \mu)=C-\alpha_{i}-\mu_{i}=0} \\ {w=\sum_{i=1}^{N} \alpha_{i} y_{i} x_{i}} \\ {\sum_{i=1}^{N} \alpha_{i} y_{i}=0} \\ {C-\alpha_{i}-\mu_{i}=0}\end{array}
$$

$$
\min _{w, b, \xi} L(w, b, \xi, \alpha, \mu)=-\frac{1}{2} \sum_{i=1}^{N} \sum_{j=1}^{N} \alpha_{i} \alpha_{j} y_{i} y_{j}\left(x_{i} \cdot x_{j}\right)+\sum_{i=1}^{N} \alpha_{i}
$$

- 极大

$$
\begin{aligned} \max _{\alpha} & -\frac{1}{2} \sum_{i=1}^{N} \sum_{j=1}^{N} \alpha_{i} \alpha_{j} y_{i} y_{j}\left(x_{i} \cdot x_{j}\right)+\sum_{i=1}^{N} \alpha_{i} \\ \text { s.t. } \quad & \sum_{i=1}^{N} \alpha_{i} y_{i}=0 \\ & C-\alpha_{i}-\mu_{i}=0 \\ & \alpha_{i} \geqslant 0 \\ & \mu_{i} \geqslant 0, \quad i=1,2, \cdots, N \end{aligned}
$$

后三条限制消去 $\mu_i$，等价为 $0 \leqslant \alpha_{i} \leqslant C$
$$
\begin{array}{ll}{\min _{\alpha}} & {\frac{1}{2} \sum_{i=1}^{N} \sum_{j=1}^{N} \alpha_{i} \alpha_{j} y_{i} y_{j}\left(x_{i} \cdot x_{j}\right)-\sum_{i=1}^{N} \alpha_{i}} \\ {\text { s.t. }} & {\sum_{i=1}^{N} \alpha_{i} y_{i}=0} \\ {} & {0 \leqslant \alpha_{i} \leqslant C, \quad i=1,2, \cdots, N}\end{array}
$$
假定解为 $\alpha^{*}=\left(\alpha_{1}^{*}, \alpha_{2}^{*}, \cdots, \alpha_{N}^{*}\right)^{\mathrm{T}}$。原始问题为凸二次规划，满足 KKT 条件
$$
\begin{array}{c}{\nabla_{w} L\left(w^{*}, b^{*}, \xi^{*}, \alpha^{*}, \mu^{*}\right)=w^{*}-\sum_{i=1}^{N} \alpha_{i}^{*} y_{i} x_{i}=0} \\ {\nabla_{b} L\left(w^{*}, b^{*}, \xi^{*}, \alpha^{*}, \mu^{*}\right)=-\sum_{i=1}^{N} \alpha_{i}^{*} y_{i}=0} \\ {\nabla_{\xi} L\left(w^{*}, b^{*}, \xi^{*}, \alpha^{*}, \mu^{*}\right)=C-\alpha^{*}-\mu^{*}=0} \\ {\alpha_{i}^{*}\left(y_{i}\left(w^{*} \cdot x_{i}+b^{*}\right)-1+\xi_{i}^{*}\right)=0} \\ {\mu_{i}^{*} \xi_{i}^{*}=0}
\\{y_{i}\left(w^{*} \cdot x_{i}+b^{*}\right)-1+\xi_{i}^{*} \geqslant 0} \\ {\xi_{i}^{*} \geqslant 0} \\ {\alpha_{i}^{*} \geqslant 0} \\ {\mu_{i}^{*} \geqslant 0, \quad i=1,2, \cdots, N}
\end{array}
$$
得 $w^{*}=\sum_{i=1}^{N} \alpha_{i}^{*} y_{i} x_{i}$，若 $\exists a_j^*,0<a_j^*<C$，则$y_{i}\left(w^{*} \cdot x_{i}+b^{*}\right)-1+\xi_{i}^{*}=0$，求得$b^{*}=y_{j}-\sum_{i=1}^{N} y_{i} \alpha_{i}^{*}\left(x_{i} \cdot x_{j}\right)$

- 结果

分离超平面
$$
w^{*} \cdot x+b^{*}=\sum_{i=1}^{N} \alpha_{i}^{*} y_{i}\left(x \cdot x_{i}\right)+b^{*}=0
$$
分类决策函数
$$
f(x)=\operatorname{sign}\left(w^{*} \cdot x+b^{*}\right)=\operatorname{sign}\left(\sum_{i=1}^{N} \alpha_{i}^{*} y_{i}\left(x \cdot x_{i}\right)+b^{*}\right)
$$

## 软间隔支持向量

$\alpha_i^* > 0$ 的实例为支持向量，$\frac{\xi_{i}}{\|w\|}$ 是间隔距离，$\xi_i = 0$  在间隔边界上，$0<\xi_i<1$ 正确分类，在间隔内，$\xi_i=1$ ，超平面上，$\xi_i > 1$ 分类错误

# 非线性支持向量分类机与核函数

![](https://raw.githubusercontent.com/LibertyDream/diy_img_host/master/img/2019-11-28_svm_kernal.png)

核技巧是指一类用线性分类解决非线性分类问题的技巧。将原空间数据映射到新空间，在新空间里用线性分类学习方法学习分类模型，具体到支持向量机，就是通过一种变换，使输入空间 $R^n$ 中的超曲面模型对应到特征空间 $\mathcal{H}$  里的超平面模型。这种变换就是核函数。

>  **核函数**:$\mathcal{X}$ 是输入空间（欧式空间$R^n$或离散空间），$\mathcal{H}$ 为特征空间（希尔伯特空间），若存在映射 $\phi(x): \mathcal{X} \rightarrow \mathcal{H}$ ，使对所有 $x,z \in \mathcal{X}$ ，函数 $K(x,z)$ 满足 $K(x, z)=\phi(x) \cdot \phi(z)$，就称 $K(x,z)$ 为核函数，$\phi(x) \cdot \phi(z)$ 为内积

这里核技巧的想法是直接定义内积运算结果，而不显式地定义映射函数。特征空间通常是高维的，高维空间下直接算 $K(x,z)$ 比较容易，通过 $\phi(x),\phi(z)$ 算 $K(x,z)$ 比较难。

注意到无论目标函数还是超平面方程，都只是关注内积 $x_i \cdot x_j$ ，用核函数代替，得到新目标函数与决策函数
$$
\begin{aligned} 
W(\alpha)&=\frac{1}{2} \sum_{i=1}^{N} \sum_{j=1}^{N} \alpha_{i} \alpha_{j} y_{i} y_{j} K\left(x_{i}, x_{j}\right)-\sum_{i=1}^{N} \alpha_{i}\\
f(x) &=\operatorname{sign}\left(\sum_{i=1}^{N_{s}} a_{i}^{*} y_{i} \phi\left(x_{i}\right) \cdot \phi(x)+b^{*}\right) \\ &=\operatorname{sign}\left(\sum_{i=1}^{N_{s}} a_{i}^{*} y_{i} K\left(x_{i}, x\right)+b^{*}\right) \end{aligned}
$$
判定一个函数是不是核函数：

1. 是定义域上的对称函数
2. 定义域内元素 $\phi(x)$ 构成的内积矩阵，即Gram矩阵 $K=\left[K\left(x_{i}, x_{j}\right)\right]_{m \times m}$，是半正定的（行列式大于等于0）

第二条验证比较费事，常用现成的几个核函数

- 多项式核函数

$$
K(x, z)=(x \cdot z+1)^{p}\\
f(x)=\operatorname{sign}\left(\sum_{i=1}^{N_{s}} a_{i}^{*} y_{i}\left(x_{i} \cdot x+1\right)^{p}+b^{*}\right)
$$

- 高斯核函数

$$
K(x, z)=\exp \left(-\frac{\|x-z\|^{2}}{2 \sigma^{2}}\right)\\
f(x)=\operatorname{sign}\left(\sum_{i=1}^{N_{s}} a_{i}^{*} y_{i} \exp \left(-\frac{\left\|x-x_{i}\right\|^{2}}{2 \sigma^{2}}\right)+b^{*}\right)
$$

核函数不仅能处理连续值，离散值也可以（连续化后求余弦相似度）

# 序列最小最优化算法（SMO）

SMO 要解决的问题是：
$$
\begin{aligned} \min _{\alpha} & \frac{1}{2} \sum_{i=1}^{N} \sum_{j=1}^{N} \alpha_{i} \alpha_{j} y_{i} y_{j} K\left(x_{i}, x_{j}\right)-\sum_{i=1}^{N} \alpha_{i} \\ \text { s.t. } & \sum_{i=1}^{N} \alpha_{i} y_{i}=0 \\ & 0 \leqslant \alpha_{i} \leqslant C, \quad i=1,2, \cdots, N \end{aligned}
$$


由上文可知，知道了 $\alpha^*$，也就知道了最终的解，所以关键是求解拉格朗日乘子上。但是乘子向量的维数肯可能很大，同时对多个变量进行最优化求解费时费力。SMO 的思想就是“拆”，通过若干次两个变量的二次规划得到全局最优，因为如果所有变量的解都满足了 KKT 条件，那么也就得到 $\alpha^*$ 了。所以有两个子任务，一是挑选变量的方法，二是二次规划求解的方法

## 二次规划求解法

假定选择了 $\alpha_1,\alpha_2$ 作为变量，其它的视为常数，那么原始问题变为
$$
\begin{array}{ll}{\min _{\alpha_{1}, \alpha_{2}}} & {W\left(\alpha_{1}, \alpha_{2}\right)=\frac{1}{2} K_{11} \alpha_{1}^{2}+\frac{1}{2} K_{22} \alpha_{2}^{2}+y_{1} y_{2} K_{12} \alpha_{1} \alpha_{2}-} \\ {} & {\left(\alpha_{1}+\alpha_{2}\right)+y_{1} \alpha_{1} \sum_{i=3}^{N} y_{i} \alpha_{i} K_{i 1}+y_{2} \alpha_{2} \sum_{i=3}^{N} y_{i} \alpha_{i} K_{i 2}} +D \\ {\text { s.t. } \quad} & {\alpha_{1} y_{1}+\alpha_{2} y_{2}=-\sum_{i=3}^{N} y_{i} \alpha_{i}=\zeta} \\ {} & {0 \leqslant \alpha_{i} \leqslant C, \quad i=1,2}\end{array}
$$
$D$ 是和两个变量无关的其他项加和的常数，$\zeta$ 是保证 $b^*$ 的常数。可以看出在约束条件下，关于 $W(\alpha_1,\alpha_2)$ 的最优化问题实质上是单变量的最优化问题（另一个通过等式约束求解）,而单变量视角下原问题就是线段上找最优点。所以分两步：

- 求无不等式约束下的最优解
- 加上约束后的最优解

以 $\alpha_2$ 为目标变量，初始可行解为$\alpha_{1}^{\text {old }}, \alpha_{2}^{\text {old }}$，最优解为$\alpha_{1}^{\text {new }}, \alpha_{2}^{\text {new }}$,考虑 $0 \leqslant \alpha_2 \leqslant C$ 前（未剪辑）的最优解为$\alpha_{2}^{\text {new,unc}}$。设计两个函数
$$
g(x)=\sum_{i=1}^{N} \alpha_{i} y_{i} K\left(x_{i}, x\right)+b\\
E_{i}=g\left(x_{i}\right)-y_{i}=\left(\sum_{j=1}^{N} \alpha_{j} y_{j} K\left(x_{j}, x_{i}\right)+b\right)-y_{i}, \quad i=1,2
$$
$E$ 是估计值与真实值的误差。由此得到解析解
$$
\alpha_{2}^{\text {new, unc }}=\alpha_{2}^{\text {old }}+\frac{y_{2}\left(E_{1}-E_{2}\right)}{\eta}\\
\eta=K_{11}+K_{22}-2 K_{12}=\left\|\Phi\left(x_{1}\right)-\Phi\left(x_{2}\right)\right\|^{2}
$$
剪辑后变成
$$
\alpha_{2}^{\text {new }}=\left\{\begin{array}{ll}{H,} & {\alpha_{2}^{\text {new, unc }}>H} \\ {\alpha_{2}^{\text {new, unc }},} & {L \leqslant \alpha_{2}^{\text {new, unc }} \leqslant H} \\ {L,} & {\alpha_{2}^{\text {new, unc }}<L}\end{array}\right.\\
\alpha_{1}^{\text {new }}=\alpha_{1}^{\text {old }}+y_{1} y_{2}\left(\alpha_{2}^{\text {old }}-\alpha_{2}^{\text {new }}\right)
$$
如果 $y_1 \neq y_2$，$L=\max \left(0, \alpha_{2}^{\text {old }}-\alpha_{1}^{\text {old }}\right), \quad H=\min \left(C, C+\alpha_{2}^{\text {old }}-\alpha_{1}^{\text {old }}\right)$

如果 $y_1=y_2$，$L=\max \left(0, \alpha_{2}^{\text {old }}+\alpha_{1}^{\text {old }}-C\right), \quad H=\min \left(C, \alpha_{2}^{\text {old }}+\alpha_{1}^{\text {old }}\right)$

## 变量选择方法

选择违背 KKT 条件最厉害的变量，因为违背的越厉害，优化后带来的提升越明显。

- 第一个变量的选择——外层循环

在给定精度 $\epsilon$ 内搜索，优先选择间隔边界上的点，即 $0 < \alpha_i < C（\xi = 0）$ 的样本点，检查它们是否满足 KKT 条件

- 第二个变量的选择——内层循环

有了第一个变量，第二个变量的选择就是要保证 $| E_1 - E_2|$ 最大。通常所有 $E_i$ 值保存在一个列表中

- 计算阈值 $b$ 和差值 $E_i$

每次优化完两个变量后都要更新这两个值



# 算法实现

- 导入相关库

In [1]:
import numpy as np
import cvxopt as opt

opt.solvers.options['show_progress'] = False # 略去优化过程信息

这里借助 `cvxopt` 凸优化库进行二次规划计算，要转换成二次规划标准形式

$$
\begin{array}{cl}{\min } & {\frac{1}{2} x^{T} P x+q^{T} x } \\ {\text {s.t.}} & {G x \leqslant h} \\ {} & {A x=b}\end{array}
$$

构建好对应的 `P、q、G、h、A、b` 就可以通过接口获取优化结果了，这里我们要优化的目标函数是带核函数的

$$
W(\alpha)=\frac{1}{2} \sum_{i=1}^{N} \sum_{j=1}^{N} \alpha_{i} \alpha_{j} y_{i} y_{j} K\left(x_{i}, x_{j}\right)-\sum_{i=1}^{N} \alpha_{i}
$$

- 硬件与版本信息

In [2]:
%load_ext watermark
%watermark -v -m -p ipywidgets,numpy,cvxopt

CPython 3.7.3
IPython 7.6.1

ipywidgets 7.5.0
numpy 1.16.4
cvxopt 1.2.0

compiler   : MSC v.1915 64 bit (AMD64)
system     : Windows
release    : 10
machine    : AMD64
processor  : Intel64 Family 6 Model 60 Stepping 3, GenuineIntel
CPU cores  : 4
interpreter: 64bit


- 核函数

线性核

In [3]:
def linear_kernel(**kwargs):
    def f(x1, x2):
        return np.inner(x1, x2)
    return f

多项式核

In [4]:
def polynomial_kernel(power, coef, **kwargs):
    def f(x1, x2):
        return (np.inner(x1, x2) + coef) ** power
    return f

高斯核（径向基核函数）

In [5]:
def rbf_kernel(gamma, **kwargs):
    def f(x1, x2):
        norm_2 = np.linalg.norm(x1 - x2) ** 2
        return np.exp(-gamma * norm_2)
    return f

In [6]:
class SVM(object):
    '''支持向量机
    
    参数：
    - - - - - 
    C：
        惩罚系数
    kernel：
        核方法
    power：
        多项式核幂指数
    coef:
        多项式核偏移项
    gamma：
        高斯核参数
    '''
    
    def __init__(self, C=1, kernel=rbf_kernel, power=4, gamma=None, coef=4):
        self.C = C
        self.kernel = kernel
        self.power = power
        self.gamma = gamma
        self.coef = coef
        self.lagr_multiplers = None  # 拉格朗日乘子
        self.support_vectors = None  # 支持向量
        self.support_vector_labels = None  # 支持向量的预测值
        self.b = None  # 截距
        
    def fit(X_train, y_train):
        
        n_samples, n_features = np.shape(X_train)
        
        
        # 默认无偏估计，比重相同
        if not self.gamma:
            self.gamma = 1 / n_features
        
               
        self.kernel = self.kernel(power = self.power, 
                                  gamma = self.gamma, 
                                  coef = self.coef)
        
        kernel_matrix = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                kernel_matrix[i][j] = self.kernel(X_train[i], X_train[j])
                
        P = opt.matrix(np.outer(y_train, y_train) * kernel_matrix, tc='d')
        q = opt.matrix(np.ones(n_samples) * -1)
        A = opt.matrix(y_train, (1, n_samples), tc='d')
        b = opt.matrix(0, tc='d')  # tc：type code， d 表示实数
        
        if not self.C:
            G = opt.matrix(np.identity(n_samples) * -1)
            h = opt.matrix(np.zeros(n_samples))
        else:
            G_max = np.identity(n_samples) * -1
            G_min = np.identity(n_samples)
            G = opt.matrix(np.vstack((G_max,G_min)))
            h_max = opt.matrix(np.zeros(n_samples))
            h_min = opt.matrix(np.ones(n_samples) * self.C)
            h = opt.matrix(np.vstack((h_max,h_min)))
        
        minimization = opt.solvers.qp(P, q, G, h, A, b)
        
        lagr_mult = np.ravel(minimization['x'])
        
        idx = lagr_mult > 1e-7
        
        self.lagr_multiplers = lagr_mult[idx]
        
        self.support_vectors = X_train[idx]
        
        self.support_vector_labels = y_train[idx]
        
        self.b = self.support_vectors_labels[0]
        for i in range(len(self.lagr_multiplers)):
            self.b -= self.lagr_multiplers[i] * self.support_vectors_labels[i] * self.kernel(self.support_vectors[i], self.support_vectors[0])
    
    def predict(self, X_test):
        
        y_pred = []
        
        for x in X_test:
            pred = 0
            for i in range(len(self.lagr_multiplers)):
                pred += self.lagr_multiplers[i] * self.support_vectors_labels[i] * self.kernel(self.support_vectors[i], x)
            pred += self.b
            y_pred.append(np.sign(pred))
        return np.array(y_pred)

---

作者：Daniel Meng

GitHub：[LibertyDream](https://github.com/LibertyDream)

博客：[明月轩](https://libertydream.github.io/)